In [ ]:
pip install rdkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 65.3 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd

from rdkit import Chem
from rdkit.Chem import AllChem
import rdkit.Chem.Fragments as f
import rdkit.Chem.rdMolDescriptors as d
from rdkit.Chem import Lipinski as l

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV

from sklearn import metrics
from sklearn.metrics import auc, roc_curve

import matplotlib.pyplot as plt

## Importing Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Filepath to shared drive (you have to add the 'Programming for Data Science' folder to your drive)
filepath = '/content/drive/MyDrive/Programming for Data Science/Assignment 4/'

# Filepath to your own drive folder and files if above doesnt work
#filepath = '/content/drive/MyDrive/Colab Notebooks/ID2214/'

In [ ]:
train_data = pd.read_csv(filepath + "training_smiles.csv")

In [ ]:
y = train_data["ACTIVE"]

In [ ]:
def get_atom_features(dataframe):
    df = dataframe.copy()

    # Features in arrays
    FRAC_CSP3 = []
    HEAVY_ATOMS = []
    MOL_WEIGHT = []
    NHOH = []
    NO = []
    ALIPH_CARB = []
    ALIPH_HETE = []
    ALIPH_RING = []
    AROM_CARB = []
    AROM_HETER = []
    AROM_RING = []
    H_ACCEPTORS = []
    H_DONORS = []
    H_ATOMS = []
    SATUR_CARBS = []
    SATUR_HETER = []
    SATUR_RING = []
    RING = []


    for row in df.index:
        smile = df.loc[row,'SMILES']
        m = Chem.MolFromSmiles(smile)

        FRAC_CSP3.append(l.FractionCSP3(m))
        HEAVY_ATOMS.append(l.HeavyAtomCount(m))
        MOL_WEIGHT.append(d.CalcExactMolWt(m))
        NHOH.append(l.NHOHCount(m))
        NO.append(l.NOCount(m))
        ALIPH_CARB.append(l.NumAliphaticCarbocycles(m))
        ALIPH_HETE.append(l.NumAliphaticHeterocycles(m))
        ALIPH_RING.append(l.NumAliphaticRings(m))
        AROM_CARB.append(l.NumAromaticCarbocycles(m))
        AROM_HETER.append(l.NumAromaticHeterocycles(m))
        AROM_RING.append(l.NumAromaticRings(m))
        H_ACCEPTORS.append(l.NumHAcceptors(m))
        H_DONORS.append(l.NumHDonors(m))
        H_ATOMS.append(l.NumHeteroatoms(m))
        SATUR_CARBS.append(l.NumSaturatedCarbocycles(m))
        SATUR_HETER.append(l.NumSaturatedHeterocycles(m))
        SATUR_RING.append(l.NumSaturatedRings(m))
        RING.append(l.RingCount(m))


    df['FRAC_CSP3'] = FRAC_CSP3
    df['HEAVY_ATOMS'] = HEAVY_ATOMS
    df['MOL_WEIGHT'] = MOL_WEIGHT
    df['NHOH'] = NHOH
    df['NO'] = NO
    df['ALIPH_CARB'] = ALIPH_CARB
    df['ALIPH_HETE'] = ALIPH_HETE
    df['ALIPH_RING'] = ALIPH_RING
    df['AROM_CARB'] = AROM_CARB
    df['AROM_HETER'] = AROM_HETER
    df['AROM_RING'] = AROM_RING
    df['H_ACCEPTORS'] = H_ACCEPTORS
    df['H_DONORS'] = H_DONORS
    df['H_ATOMS'] = H_ATOMS
    df['SATUR_CARBS'] =  SATUR_CARBS
    df['SATUR_HETER'] = SATUR_HETER
    df['SATUR_RING'] = SATUR_RING
    df['RING'] = RING

    df_atom = df.drop(["INDEX", 'SMILES'], axis=1)
    #display(df_atom.head())

    return df_atom

In [ ]:
def get_morgan_features(dataframe):

    df = dataframe.copy()
    MORGAN = []

    for row in df.index:
        smile = df.loc[row,'SMILES']
        m = Chem.MolFromSmiles(smile)

        MORGAN.append(np.array(AllChem.GetMorganFingerprintAsBitVect(m,2,nBits=124)))

    df_morgan = pd.DataFrame.from_records(MORGAN, columns=['f{}'.format(i) for i in range(MORGAN[0].size)])
    #display(df_morgan)

    return df_morgan

In [ ]:
def get_complete_features(dataframe):
    df_complete = pd.merge(df_atom, df_morgan, left_index=True, right_index=True)
    display(df_complete)

    return df_complete

In [ ]:
df_atom = get_atom_features(train_data).drop("ACTIVE", axis = 1)

df_morgan = get_morgan_features(train_data)

df_complete = pd.merge(df_atom, df_morgan, left_index=True, right_index=True)

[18:15:44] WARNING: not removing hydrogen atom without neighbors
[18:16:34] WARNING: not removing hydrogen atom without neighbors


## Split and Normalization

In [ ]:
def split(data, y):
    x_train, x_val, y_train, y_val = train_test_split(data, y, test_size=0.2, random_state=1, stratify = y)
    return x_train, x_val, y_train, y_val


# features to normalize
norm_features = list(df_atom.columns)

def scale(x_train, x_val, label_list):
    train = x_train.loc[:, label_list]
    val = x_val.loc[:, label_list]

    scaler = MinMaxScaler()

    train = scaler.fit_transform(train)
    val = scaler.transform(val)

    x_train.loc[:, label_list] = np.array(train)
    x_val.loc[:, label_list] = np.array(val)

    return x_train, x_val

## Models

In [ ]:
def mlp(x_train, x_val,  y_train, y_val, params=None):

    if params == None:
        model = MLPClassifier(random_state = 10)
    else:
        model = MLPClassifier(random_state = 10, **params)

    model.fit(x_train, y_train)
    prediction = model.predict_proba(x_val)
    fpr, tpr, thre = metrics.roc_curve(y_val,prediction[:, 1])
    auc_ = metrics.auc(fpr,tpr)
    return model, auc_, prediction


def random_forest(x_train, x_val,  y_train, y_val, params=None):

    if params == None:
        model = RandomForestClassifier(class_weight='balanced', random_state = 10)
    else:
        model = RandomForestClassifier(random_state = 10, **params)

    model.fit(x_train, y_train)
    prediction = model.predict_proba(x_val)
    fpr, tpr, thre = metrics.roc_curve(y_val,prediction[:, 1])
    auc_ = metrics.auc(fpr,tpr)
    return model, auc_, prediction

def logistic_regression(x_train, x_val,  y_train, y_val, params=None):

    if params == None:
        model = LogisticRegression(class_weight='balanced', max_iter=2000)
    else:
        model = LogisticRegression(max_iter=2000, **params)

    model.fit(x_train, y_train)
    prediction = model.predict_proba(x_val)
    fpr, tpr, thre = metrics.roc_curve(y_val,prediction[:, 1])
    auc_ = metrics.auc(fpr,tpr)

    return model, auc_, prediction


# basic models results, without tuning
def baseline(train, y, norm=False, norm_list=norm_features):
    x_train, x_val, y_train, y_val = split(train, y)

    if norm == True:
        x_train, x_val = scale(x_train, x_val, norm_list)

    lr, lr_auc, lr_prediction = logistic_regression(x_train,x_val,y_train, y_val)
    nn, nn_auc, nn_prediction = mlp(x_train,x_val,y_train, y_val)
    rf, rf_auc, rf_prediction = random_forest(x_train,x_val,y_train, y_val)
    baseline_r = [lr_auc, nn_auc, rf_auc]

    print("lr auc without tuning: " + str(lr_auc))
    print("nn auc without tuning: " + str(nn_auc))
    print("rf auc without tuning: " + str(rf_auc))

    return baseline_r

In [ ]:
# baseline our AUC
baseline_rows = ["basic", "m_f", "basic+m_f"]
baseline_columns = ["logistic_reg", "neural_net", "random_forest"]

r1 = baseline(df_atom, y)
r2 = baseline(df_morgan, y)
r3 = baseline(df_complete, y)

baseline_r = [r1, r2, r3]
r_df = pd.DataFrame(baseline_r, columns=baseline_columns, index=baseline_rows)
display(r_df)

lr auc without tuning: 0.7622811796650594
nn auc without tuning: 0.6815361534453344
rf auc without tuning: 0.7041591527394834
lr auc without tuning: 0.7309163549812988
nn auc without tuning: 0.7344023122135402
rf auc without tuning: 0.7308478085142756
lr auc without tuning: 0.8071488634752628
nn auc without tuning: 0.8001419744150545
rf auc without tuning: 0.7881364300999973


,logistic_reg,neural_net,random_forest
basic,0.762281,0.681536,0.704159
m_f,0.730916,0.734402,0.730848
basic+m_f,0.807149,0.800142,0.788136


## Tuning Parameters

In [ ]:
def hyper_tuning(x_train,  y_train, model, kf, params):
    grid_search = GridSearchCV(model, param_grid=params, cv=kf, scoring='roc_auc')
    grid_search.fit(x_train, y_train)
    best_params = grid_search.best_params_

    return best_params, grid_search

### Neural Network

In [ ]:
nn_params = {
    'hidden_layer_sizes': [(10,),(20,)],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.005],
}

kf = StratifiedKFold(n_splits=5)

x_train, x_val, y_train, y_val = split(df_complete, y)
#x_train, x_val = scale(x_train, x_val, norm_features)

nn, auc, pred = mlp(x_train, x_val,  y_train, y_val)

In [ ]:
best_params_nn, grid_search_nn = hyper_tuning(x_train, y_train, model=nn, kf=kf, params=nn_params)

In [ ]:
nn_updated, nn_auc_updated, nn_prediction_updated = mlp(x_train, x_val,  y_train, y_val, best_params_nn)

In [ ]:
results_nn = pd.DataFrame.from_dict(grid_search_nn.cv_results_)
display(results_nn)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_hidden_layer_sizes,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,23.384371,3.008133,0.029954,0.002572,0.0001,"(10,)",sgd,"{'alpha': 0.0001, 'hidden_layer_sizes': (10,),...",0.500000,0.500000,0.305739,0.500000,0.317121,0.424572,0.092450,6
1,15.239444,5.045083,0.031452,0.002007,0.0001,"(10,)",adam,"{'alpha': 0.0001, 'hidden_layer_sizes': (10,),...",0.809602,0.807770,0.779847,0.816104,0.788608,0.800386,0.013763,4
2,26.620585,3.516794,0.031337,0.002271,0.0001,"(20,)",sgd,"{'alpha': 0.0001, 'hidden_layer_sizes': (20,),...",0.500000,0.499980,0.500000,0.346986,0.407830,0.450959,0.063061,5
3,21.160705,5.039763,0.031862,0.000355,0.0001,"(20,)",adam,"{'alpha': 0.0001, 'hidden_layer_sizes': (20,),...",0.811137,0.810643,0.793637,0.809819,0.783481,0.801743,0.011242,2
4,22.385894,2.724792,0.033680,0.010800,0.005,"(10,)",sgd,"{'alpha': 0.005, 'hidden_layer_sizes': (10,), ...",0.500000,0.500000,0.305935,0.500000,0.316263,0.424440,0.092600,7
5,15.034114,4.494839,0.030219,0.000807,0.005,"(10,)",adam,"{'alpha': 0.005, 'hidden_layer_sizes': (10,), ...",0.808806,0.807827,0.778360,0.818079,0.790872,0.800789,0.014242,3
6,24.606127,3.361476,0.036000,0.011927,0.005,"(20,)",sgd,"{'alpha': 0.005, 'hidden_layer_sizes': (20,), ...",0.500040,0.361503,0.500000,0.312078,0.410844,0.416893,0.074714,8
7,18.338594,6.311525,0.034361,0.005301,0.005,"(20,)",adam,"{'alpha': 0.005, 'hidden_layer_sizes': (20,), ...",0.813169,0.814304,0.797887,0.813300,0.793241,0.806380,0.008961,1


### Random Forest

In [ ]:
rf_params = {
    'n_estimators': [200, 300, 400],
    'class_weight': ['balanced'],
    'max_depth': [30, 50]
}

kf = StratifiedKFold(n_splits=5)

x_train, x_val, y_train, y_val = split(df_complete, y)
#x_train, x_val = scale(x_train, x_val, norm_features)

rf, auc, pred = random_forest(x_train, x_val,  y_train, y_val)

In [ ]:
best_params_rf, grid_search_rf = hyper_tuning(x_train, y_train, model=rf, kf=kf, params=rf_params)

rf_updated, rf_auc_updated, rf_prediction_updated = random_forest(x_train, x_val,  y_train, y_val, best_params_rf)

In [ ]:
results_rf = pd.DataFrame.from_dict(grid_search_rf.cv_results_)
display(results_rf)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_class_weight,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,28.017805,0.368317,1.041497,0.037619,balanced,30,200,"{'class_weight': 'balanced', 'max_depth': 30, ...",0.826352,0.833752,0.815276,0.810134,0.807628,0.818629,0.009927,6
1,41.921076,0.947340,1.458125,0.096109,balanced,30,300,"{'class_weight': 'balanced', 'max_depth': 30, ...",0.832153,0.843807,0.831558,0.823132,0.824610,0.831052,0.007326,3
2,55.589790,0.928359,1.793666,0.214467,balanced,30,400,"{'class_weight': 'balanced', 'max_depth': 30, ...",0.846571,0.856609,0.836717,0.837433,0.832647,0.841995,0.008609,1
3,27.885438,0.344131,1.037383,0.196137,balanced,50,200,"{'class_weight': 'balanced', 'max_depth': 50, ...",0.817905,0.843936,0.813223,0.813228,0.813694,0.820397,0.011900,5
4,41.273828,0.829382,1.594731,0.206735,balanced,50,300,"{'class_weight': 'balanced', 'max_depth': 50, ...",0.828664,0.858879,0.821839,0.824553,0.816409,0.830069,0.014945,4
5,54.300440,0.645166,1.954595,0.098114,balanced,50,400,"{'class_weight': 'balanced', 'max_depth': 50, ...",0.841593,0.864723,0.826077,0.828072,0.820385,0.836170,0.015883,2


### Logistic Regression

In [ ]:
lg_params = {
    'fit_intercept': [True, False],
    'class_weight': ['balanced'],
    'penalty': ["l2", "none"]
}

kf = StratifiedKFold(n_splits=5)

x_train, x_val, y_train, y_val = split(df_complete, y)
#x_train, x_val = scale(x_train, x_val, norm_features)

lg, auc, pred = logistic_regression(x_train, x_val,  y_train, y_val)

In [ ]:
best_params_lg, grid_search_lg = hyper_tuning(x_train, y_train, model=lg, kf=kf, params=lg_params)

lg_updated, lg_auc_updated, lg_prediction_updated = logistic_regression(x_train, x_val, y_train, y_val, best_params_lg)

In [28]:
results_lg = pd.DataFrame.from_dict(grid_search_lg.cv_results_)
display(results_lg)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_class_weight,param_fit_intercept,param_penalty,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,27.974939,4.804206,0.025408,0.007494,balanced,True,l2,"{'class_weight': 'balanced', 'fit_intercept': ...",0.821531,0.821440,0.812926,0.824200,0.797116,0.815443,0.009919,1
1,27.201831,3.098084,0.027650,0.010922,balanced,True,none,"{'class_weight': 'balanced', 'fit_intercept': ...",0.821634,0.821395,0.812349,0.823661,0.797049,0.815217,0.009889,2
2,28.329747,2.219037,0.023202,0.001861,balanced,False,l2,"{'class_weight': 'balanced', 'fit_intercept': ...",0.819289,0.820194,0.810576,0.823648,0.795217,0.813785,0.010234,4
3,25.446342,5.118467,0.030932,0.009302,balanced,False,none,"{'class_weight': 'balanced', 'fit_intercept': ...",0.818838,0.820343,0.811381,0.823223,0.795403,0.813838,0.010013,3


## Test Data

In [29]:
# output the results on test dataset
test_data = pd.read_csv(filepath+"test_smiles.csv")

# add features
test_data = pd.merge(get_atom_features(test_data), get_morgan_features(test_data), left_index=True, right_index=True)

[21:29:35] WARNING: not removing hydrogen atom without neighbors
[21:29:52] WARNING: not removing hydrogen atom without neighbors


## Best Model

In [30]:
#Choose the best parameters

final_params = best_params_rf# insert the best

x_train, x_val, y_train, y_val = split(df_complete, y)
#x_train, x_val = scale(x_train, x_val, norm_list)

In [31]:
final_model, final_auc, final_val_prediction = random_forest(x_train, x_val,  y_train, y_val, final_params)

print("best params: " + str(final_params))
print("final auc on val data: " + str(round(final_auc, 4)))

final_results = final_model.predict_proba(test_data)
print(final_results)

best params: {'class_weight': 'balanced', 'max_depth': 30, 'n_estimators': 400}
final auc on val data: 0.8543
[[0.99   0.01  ]
 [0.9925 0.0075]
 [0.9825 0.0175]
 ...
 [0.9925 0.0075]
 [0.995  0.005 ]
 [1.     0.    ]]
